<a href="https://colab.research.google.com/github/SSESLab/Campus-Decision-Tool/blob/master/Data_Driven_Building_Energy_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation of helper libraries

In [0]:
#Google Colab Helper Files
from google.colab import files
!pip install --upgrade -q pygsheets

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import math
from numpy import *
from mpl_toolkits import mplot3d
from datetime import time
from datetime import date
import datetime
from google.colab import files
from datetime import timedelta  

#Import non-linear equation solver
from scipy.optimize import fsolve


import pandas as pd

# Model Input Data

In [0]:
##Energy Costs
Central_Heating_rate=0.015407 #$/kWh heating, UofU gave 4.5 #$/MMBTU, 1 MMbtu=293 kWh, 4.5/293=0.015407
Central_Cooling_rate=0.03 #$/kWh cooling, UofU gave 8.80 #$/MMBTU, 1 MMbtu=293 kWh, 8.8/293=0.03
Electricity_rate=0.065 #$/kWh

##General Constants
gc=9.81 #m/s^2

# Import Template Files From Github


In [0]:
# Import weather file
url_weather = 'https://raw.githubusercontent.com/SSESLab/Campus-Decision-Tool/master/SLC%20Weather%20AMY%20KSLC.csv'
weatherfile = pd.read_csv(url_weather)

# Import cooling File
url_cooling='https://raw.githubusercontent.com/SSESLab/Campus-Decision-Tool/master/Normalized_cooling_classroom_auditorium.csv'
hourly_load_cool= pd.read_csv(url_cooling)
#Import heating file
url_heating='https://raw.githubusercontent.com/SSESLab/Campus-Decision-Tool/master/Normalized_heating_classroom_auditorium.csv'
hourly_load_heat= pd.read_csv(url_heating)
#Import occupancy file
url_occupancy= 'https://raw.githubusercontent.com/SSESLab/Campus-Decision-Tool/master/Normalized_occupancy_classroom_auditorium.csv'
hourly_Occupancy=pd.read_csv(url_occupancy)

#Define building size
Building_Size=5000 #Generic building size

#Define all files data to datetime format


In [4]:
#hourly_load_cool.info()
#hourly_load_heat.info()
#hourly_Occupancy.info()
hourly_load_cool.set_index('Date')
hourly_load_heat.set_index('Date')
hourly_Occupancy.set_index('Date')

,Unnamed: 0,Occupancy
Date,,
2018-01-01 00:00:00,0,0.200000
2018-01-01 01:00:00,1,0.200000
2018-01-01 02:00:00,2,0.200000
2018-01-01 03:00:00,3,0.200000
2018-01-01 04:00:00,4,0.200000
...,...,...
2018-12-31 20:00:00,8755,0.735294
2018-12-31 21:00:00,8756,0.726891
2018-12-31 22:00:00,8757,0.718487


# Psychrometric Functions

In [0]:
def Psych(DB_air_in,RH,p_total):

# Psychrometric properties for moist air
# provided dry bulb temperature (Celsius), relative humidity (#),
# and total atmospheric pressure (kPa)



    #{Calculating air properties}#
    #{Coefficient values from ASHRAE fundamentals handbook section 1.12}#
    c1=-5.6745359e+03
    c2=6.3925247e+00
    c3=-9.6778430e-03
    c4=6.2215701e-07
    c5=2.0747825e-09
    c6=-9.4840240e-13
    c7=4.1635019e00
    c8=-5.8002206e+03
    c9=1.3914993e+00
    c10=-4.8640239e-02
    c11=4.1764768e-05
    c12=-1.4452093e-08
    c13=6.5459673e00
    T=DB_air_in+273.15

    if DB_air_in==0:
        pws=math.exp(c8/T+c9+c10*T+c11*T**2+c12*T**3+c13*math.log(T)) #{Pa saturation pressure, EQN 6}#
        pw=0.01*RH*pws #{partial pressure of water vapor Eqn 24}#
        W_in=0.621945*pw/(p_total-pw) #{humidity ratio Eqn 22}#
        Ws=0.621945*pws/(p_total-pws) #{moist air saturation ratio Eqn 23}#
        h_air_inlet=1.006*DB_air_in+W_in*(2501+1.86*DB_air_in) #{kj/kg, Eqn 32}#
    if DB_air_in>0:
        pws=math.exp(c8/T+c9+c10*T+c11*T**2+c12*T**3+c13*math.log(T)) #{Pa saturation pressure, EQN 6}#
        pw=0.01*RH*pws #{partial pressure of water vapor Eqn 24}#
        W_in=0.621945*pw/(p_total-pw) #{humidity ratio Eqn 22}#
        Ws=0.621945*pws/(p_total-pws) #{moist air saturation ratio Eqn 23}#
        h_air_inlet=1.006*DB_air_in+W_in*(2501+1.86*DB_air_in) #{kj/kg, Eqn 32}#
    if DB_air_in<0:
        pws=math.exp(c1/T+c2+c3*T+c4*T**2+c5*T**3+c6*T**4+c7*math.log(T)) #{Pa saturation pressure, EQN 5}#
        pw=0.01*RH*pws #{partial pressure of water vapor Eqn 24}#
        W_in=0.621945*pw/(p_total-pw) #{humidity ratio Eqn 22}#
        Ws=0.621945*pws/(p_total-pws) #{moist air saturation ratio Eqn 23}#
        h_air_inlet=1.006*DB_air_in+W_in*(2501+1.86*DB_air_in) #{kj/kg, Eqn 32}#

#Wet Bulb emperical equation from RH and DB: Wet-Bulb Temperature from
#Relative Humidity and Air Temperature, Roland Stull https://doi.org/10.1175/JAMC-D-11-0143.1
    WB_air_in=DB_air_in*math.atan(0.151977*(RH+8.313659)**0.5)+math.atan(DB_air_in+RH)-math.atan(RH-1.676331)+0.00391838*RH**(3/2)*math.atan(0.023101*RH)-4.686035

    enthalpy = h_air_inlet # Convert to kg
    hum_ratio = W_in*1000 #Converts to g/kg
    wetbulb = WB_air_in
    Psat = pws
   
    return enthalpy, hum_ratio, wetbulb, Psat
  
#Calculate RH
def RH_calc(W_kg,H,p_total):

# Psychrometric properties for moist air
# provided dry bulb temperature (Celsius), relative humidity (#),
# and total atmospheric pressure (kPa)


    W=W_kg/1000 #convert to kg/kg
    #{Calculating air properties}#
    #{Coefficient values from ASHRAE fundamentals handbook section 1.12}#
    c1=-5.6745359e+03
    c2=6.3925247e+00
    c3=-9.6778430e-03
    c4=6.2215701e-07
    c5=2.0747825e-09
    c6=-9.4840240e-13
    c7=4.1635019e00
    c8=-5.8002206e+03
    c9=1.3914993e+00
    c10=-4.8640239e-02
    c11=4.1764768e-05
    c12=-1.4452093e-08
    c13=6.5459673e00
    DB_air_in=(H-W*2501)/(1.006+W*1.86)
    T=DB_air_in+273.15

        

    pws=math.exp(c8/T+c9+c10*T+c11*T**2+c12*T**3+c13*math.log(T)) #{Pa saturation pressure, EQN 6}#
    Ws=0.621945*pws/(p_total-pws) #{moist air saturation ratio Eqn 23}#
    pw=p_kPa*W/(0.62+W) #eqn. 22

    
    RH=(pw/(pws))*100 #Eqn 24

    
    
    return RH, DB_air_in

# Function Declaration for VAV and Central Plant

System 1 - function descriptions


In [0]:
def Chiller(Q_cooling_Load): ##Chiller Function ******Need to figure out if I keep condenser water constant or not
  #cpw=4.19 #kj/kg C
  #Q_chiller=m_dot_chw*cpw*(T_chwr-T_chws) #kWh
  NPLV=0.1017 #0.357 kW per Ton; 1 Ton = 3.51 kW = 0.1017 kW/kW - Per SW Chiller plant 
  
  P_chiller=Q_cooling_Load*NPLV
  return(P_chiller)


##Cooling Unit - Assuming air handler with a heating coil for meeting minimum air temp requirements

def ElectricBoiler(m_dot_hw,T_hws,T_hwr):
  cp_w=4.19 #kj/kgC
  P_boiler=m_dot_hw*cp_w*(T_hws-T_hwr) #kW
  return(P_boiler)

def Airhandler(AHU_Mode, V_dot_SA, m_dot_SA, Q_coil,DB_ma, T_chws,T_chwr,T_hws,T_hwr,T_min_SA,Occupancy):
  #AHU Constants
  c1=2.4292  #From Wang, 2004
  c2=0.4978 #From Wang
  c3=0.8     #From Wang
  eff_pump=0.58 #General
  H_chw=33 #meters, assumption to start
  H_hw=33 #meters, assumption to start
  dp_SA=373.623 #Pa, Assuming 1.5" WC
  eff_fan=0.4 #Fan efficiency
  
  if AHU_Mode == 'cool':
    Q_coil=Q_coil
    m_dot_chw=(m_dot_SA)/((((DB_ma-T_chws)*(c1*m_dot_SA**(c3))/Q_coil)-1)/c2)**(1/c3) ##Wang, Yaowen, et al. "An engineering model of coils and heat exchangers for HVAC system simulation and optimization." balance 1.
    P_pump_chw=(m_dot_chw*gc*H_chw/(eff_pump*1000)) #Single pump kWh, pump efficiency can be defined by a curve fitting; Use manufacturer website; Head will need to be an approximation
    P_fan_ahu_full=V_dot_SA*dp_SA/(eff_fan*1000) #kWh, Single fan, fan efficiency can be defined by a curve fitting; Use manufacturer website; Head will need to be an approximation, Janna
    P_fan_ahu=P_fan_ahu_full*(Occupancy**3)
    m_dot_hw=0
    P_pump_hw=0
    
  if AHU_Mode=='heat':
    Q_coil=m_dot_SA*1.005*(T_min_SA-DB_ma)*3600 #kW
    m_dot_hw=(m_dot_SA)/((((T_hws-DB_ma)*(c1*m_dot_SA**(c3))/Q_coil)-1)/c2)**(1/c3) ##Wang, Yaowen, et al. "An engineering model of coils and heat exchangers for HVAC system simulation and optimization." balance 1.
    P_pump_hw=(m_dot_hw*gc*H_hw/(eff_pump*1000))  #Single pump kWh, pump efficiency can be defined by a curve fitting; Use manufacturer website; Head will need to be an approximation
    P_fan_ahu_full=V_dot_SA*dp_SA/(eff_fan*1000) #kWh, Single fan, fan efficiency can be defined by a curve fitting; Use manufacturer website; Head will need to be an approximation, Janna
    P_fan_ahu=P_fan_ahu_full*Occupancy**3
    m_dot_chw=0
    P_pump_chw=0
  
  return(m_dot_chw, P_fan_ahu, P_pump_chw,m_dot_hw,P_pump_hw,Q_coil)

##Indoor units at rooms - Need indoor units equal to number of rooms.  Turn on/off from occupancy schedule, hot water re-heat
def VAV_terminal(V_dot_SA, m_dot_SA, Q_coil,T_hws,DB_ma,Occupancy):
  # Use Varitrane Parallel fan units with hot water coil  Fan Size 02SQ

  ##Heating Load heat transfer - Q_coil = room load
  #Cooling Coil Model - An Engineering Model of Coils and Heat Exchangers for HVAC System Simulation and Optimization, Wang
  c1=2.4292  #From Wang need to update, 2004
  c2=0.4978 #From Wang need to update
  c3=0.8     #From Wang need to update
  eff_pump=0.58 #General, need to make a function of flow
  H_hw=33 #meters, assumption to start
  dp_SA=373.623 #Pa, Assuming 1.5" WC
  eff_fan=0.4 #Fan efficiency

  m_dot_hw=(m_dot_SA)/((((T_hws-DB_ma)*(c1*m_dot_SA**(c3))/Q_coil)-1)/c2)**(1/c3) ##Wang, Yaowen, et al. "An engineering model of coils and heat exchangers for HVAC system simulation and optimization." balance 1.2 (2002): 3.

  P_pump_hw=(m_dot_hw*gc*H_hw/(eff_pump*1000))  #Single pump kWh, pump efficiency can be defined by a curve fitting; Use manufacturer website; Head will need to be an approximation
  P_fan_VAV_full=V_dot_SA*dp_SA/(eff_fan*1000) #kWh, Single fan, fan efficiency can be defined by a curve fitting; Use manufacturer website; Head will need to be an approximation, Janna
  #P_fan_VAV=P_fan_VAV_full*(Occupancy**3)
  P_fan_VAV=0
  
  return(P_pump_hw, P_fan_VAV,m_dot_hw)




# **System 1 - Annual energy function**

In [6]:
#Define Results Database
  columns=['P_fan_ahu','P_pump_chw','P_pump_hw','P_pump_VAV','DB_ma','m_dot_chw','m_dot_hw','m_dot_hw_VAV', 'P_chiller','Cooling_Load-kWh','Central_Cooling_Cost','Electricity_Cost','Central_Heating_Cost','Q_central_heat','Q_AHU','Q_VAV']
  results = pd.DataFrame(index=pd.RangeIndex(start=0, stop=8760, step=1), columns=columns)
  #Building_Size=8454.1766 #Square footage of FASB = 91,000 Ft^2
  #Building_Size=11148 #Square Meter, 120,000 SF per wing
  P_fan_VAV=0
  P_pump_hw=0

#General Constants
T_chws=3.33 #Chilled Water plant runs at 38F
Min_OA_percentage=.15 #Assumption
den_air=1.0228 #m^3/kg
T_chwr=11.67 #Chilled water plant returns around 11.67 C
T_hws=68.33 #Hot Water supply to building after HX, using Kennecot as example
T_hwr=62.78 #Hot Water return building after HX, using Kennecot as example
T_min_SA=12.78
    
#Space conditions
DB_RA=22.22 #Celcius
RH_RA=0.5 #%
H_RA=48.11 #kj/kg
W_RA=10.14 #kg/kg  


  for x in range(0,8760):
    #Get Date from WeatherFile
    step_date=weatherfile.loc[x,'Date']
    #Assigns weather conditions at this instance of time
    RH_OA=weatherfile.loc[x,'RH']
    DB_OA=weatherfile.loc[x,'T_DB']
    p_kPa=weatherfile.loc[x,'Atm_Press']
    (H_OA,W_OA,T_WB_OA,P_SAT_OA)=Psych(DB_OA,RH_OA,p_kPa)
    
    #Assign Cooling Load at this instanace of time
    hourly_load_cool = hourly_load_cool.set_index(hourly_load_cool['Date'])
    if (step_date in hourly_load_cool.index):
      Step=hourly_load_cool.loc[step_date]
      #Q_cooling_coil=Step[1]*Building_Size
      Q_cooling_coil=Step['Cooling_Load']*Building_Size #Finds cooling load as a function of building Size

    
    else:
      Q_cooling_coil=0 #If load database doesn't have a matching date return 0
    
    #Assign Heating Load at this instance of time
    hourly_load_heat = hourly_load_heat.set_index(hourly_load_heat['Date'])
    if (step_date in hourly_load_heat.index):
      Step=hourly_load_heat.loc[step_date]
      Q_heating_coil=Step['Heating_Load']*Building_Size #Finds heating load as a function of building Size
      if Q_heating_coil<0: #Heating load can only be positive, this eliminates negative heating loads
        Q_heating_coil=0
    else:
      Q_heating_coil=0 #If load database doesn't have a matching date return 0
    ##Ignore heating load if DB_RA is less than DB_OA
    if DB_RA<DB_OA:
      Q_heating_coil=0  
    
    #Assign Occupancy
    hourly_Occupancy= hourly_Occupancy.set_index(hourly_Occupancy['Date'])
    if (step_date in hourly_Occupancy.index):
      Occupancy=hourly_Occupancy.at[step_date,'Occupancy']
    else:
      Occupancy=0.2 #Minimum setting    
    
    #Assign airflow
    V_dot_SA=Building_Size*(0.005078) #m^3/s, Assuming 1 CFM per ft^2
    m_dot_SA=V_dot_SA/den_air
    m_dot_RA=m_dot_SA*(1-Min_OA_percentage)
    m_dot_OA=m_dot_SA-m_dot_RA
    
    #Calculate mixed air temp
    H_mixed=(m_dot_RA*H_RA+m_dot_OA*H_OA)/(m_dot_RA+m_dot_OA)
    W_mixed=(m_dot_RA*W_RA+m_dot_OA*W_OA)/(m_dot_RA+m_dot_OA)
    DB_ma=(H_mixed-W_mixed*2.501)/(1.006+1.86*W_mixed/1000)
    
    #Set AHU in Heating or cooling mode
    if DB_ma>T_min_SA:
      AHU_Mode='cool'
      Q_AHU_heat=0 #AHU can't be in heating and cooling
      P_fan_VAV=0
      P_pump_VAV=0
      (m_dot_chw, P_fan_ahu, P_pump_chw,m_dot_hw,P_pump_hw,Q_AHU_cool)=Airhandler(AHU_Mode, V_dot_SA, m_dot_SA, Q_cooling_coil,DB_ma, T_chws,T_chwr,T_hws,T_hwr,T_min_SA,Occupancy)
    else:
      AHU_Mode='heat'
      Q_AHU_cool=0 #AHU can't be in heating and cooling
      (m_dot_chw, P_fan_ahu, P_pump_chw,m_dot_hw,P_pump_hw,Q_AHU_heat)=Airhandler(AHU_Mode, V_dot_SA, m_dot_SA, Q_heating_coil,DB_ma, T_chws,T_chwr,T_hws,T_hwr,T_min_SA,Occupancy)
    
    #Run VAV boxes re-heat coil as required to heat building
    if DB_RA>DB_OA:
      Q_VAV=Q_heating_coil-Q_AHU_heat #This avoids from double counting the heating requirement at AHU and VAV re-heat
      (P_pump_VAV, P_fan_VAV,m_dot_hw_VAV)=VAV_terminal(V_dot_SA, m_dot_SA, Q_heating_coil,T_hws,DB_ma,Occupancy)
      test=1
    else:
      Q_VAV=0
      P_fan_VAV=0
      P_pump_VAV=0



    #Calculate Chiller energy
    if AHU_Mode=='cool':
      (P_chiller)=Chiller(Q_AHU_cool)

    #Calculate Total Heating required from central plant
    Q_central_heat=Q_VAV+Q_AHU_heat
    
    #Calculate Costs per hour
    Total_Electricty=P_fan_ahu+P_pump_chw+P_pump_hw+P_pump_VAV
    Electricity_Cost=Total_Electricty*Electricity_rate

    Central_Cooling_Cost=Q_cooling_coil*Central_Cooling_rate
    Central_Heating_Cost=Q_central_heat*Central_Heating_rate
    
    #Write results to database
    results.iloc[x, results.columns.get_loc('P_fan_ahu')] = P_fan_ahu

    results.iloc[x, results.columns.get_loc('P_pump_chw')] = P_pump_chw
    results.iloc[x, results.columns.get_loc('P_pump_hw')] = P_pump_hw
    results.iloc[x, results.columns.get_loc('P_pump_VAV')] = P_pump_VAV
    results.iloc[x, results.columns.get_loc('DB_ma')] = DB_ma
    results.iloc[x, results.columns.get_loc('m_dot_chw')] = m_dot_chw
    results.iloc[x, results.columns.get_loc('m_dot_hw')] = m_dot_hw
    results.iloc[x, results.columns.get_loc('m_dot_hw_VAV')] = m_dot_hw_VAV
    results.iloc[x, results.columns.get_loc('P_chiller')] = P_chiller
    results.iloc[x, results.columns.get_loc('Cooling_Load-kWh')] = Q_cooling_coil
    results.iloc[x, results.columns.get_loc('Central_Cooling_Cost')] = Central_Cooling_Cost
    results.iloc[x, results.columns.get_loc('Electricity_Cost')] = Electricity_Cost
    results.iloc[x, results.columns.get_loc('Central_Heating_Cost')] = Central_Heating_Cost
    results.iloc[x, results.columns.get_loc('Q_central_heat')] = Q_central_heat
    results.iloc[x, results.columns.get_loc('Q_AHU')] = Q_AHU_heat
    results.iloc[x, results.columns.get_loc('Q_VAV')] = Q_VAV

    
    
  
  results.to_csv('results.csv')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:62: RuntimeWarning: divide by zero encountered in double_scalars


# **System #2 Air Source VRF and Water Source VRF Functions**

In [0]:
def VRF_Coil(WB_RA, T_cond): #https://energyplus.net/sites/default/files/pdfs_v8.3.0/EngineeringReference.pdf, page 11
  TotCapTempModFac=a+b*WB_RA+c*WB_RA**2+d*T_cond+e*WB_RA**2+f*WB_RA*T_cond
  ff=m_dot_SA_coil/m_dot_SA_rated
  TotCapFlowModFac=a+b*ff+c*ff**2+d*ff**3
  Q_total=Q_ref*TotCapTempModFac*TotCapFlowModFac
  
  return(Q_total)

def VRF_Condenser(DB_OA, WB_RA,Q_coil,mode): #Raustad #https://bigladdersoftware.com/epx/docs/8-7/engineering-reference/variable-refrigerant-flow-heat-pumps.html; #https://energyplus.net/sites/default/files/pdfs_v8.3.0/EngineeringReference.pdf page 625 to 637
  ## Read VRF Condenser coefficient parameters
  (COP_cool_ref,COP_heat_ref,Q_cool_rated,Q_heat_rated,IEER_rated,x_oa_cool,x_ft_cool,x_EIR_cool,x_PLR_cool,x_ffplr,x_oa_heat,x_ft_heat,x_EIR_heat,x_PLR_heat)=VRF_288()
  ##Heat recovery
  HRCapMod_cooling=0.91

  ##Cooling calcs 
  if mode == 'cool':  
    DB_OA_Boundary_cool=x_oa_cool[0]+x_oa_cool[1]*WB_RA+x_oa_cool[2]*WB_RA**2+x_oa_cool[3]*WB_RA**3
    #DB_OA_Boundary determines if high or low curves are used; If DB_OA < DB_OA_Boundary then use low temp. region performance else use high temp region performance
  
    CAPFT_HP_cool=x_ft_cool[0]+x_ft_cool[1]*WB_RA+x_ft_cool[2]*WB_RA**2+x_ft_cool[3]*DB_OA+x_ft_cool[4]*DB_OA**2+x_ft_cool[5]*WB_RA*DB_OA
    Q_cool_available=Q_cool_rated*CAPFT_HP_cool
    PLR_cool=Q_coil/Q_cool_available
    EIRFT_HP_cool=x_EIR_cool[0]+x_EIR_cool[1]*WB_RA+x_EIR_cool[2]*WB_RA**2+x_EIR_cool[3]*DB_OA+x_EIR_cool[4]*DB_OA**2+x_EIR_cool[5]*WB_RA*DB_OA
    EIRFPLR_cool=x_PLR_cool[0]+x_PLR_cool[1]+x_PLR_cool[2]*PLR_cool**2+x_PLR_cool[3]*PLR_cool**3
    CR_rated_cool=Q_coil/Q_cool_rated
  
    #CR_cooling_correction=a_cr+b_cr*CR_rated+c_cr*CR_rated**2+d_cr*CR_rated**3 ##Raustad page 33
    CR_cooling_corection=1 ##CR_cooling_correction is for when the number of indoor unit connected load exceeds the outdoor units total load, IGNORING FOR NOW
    HPRTF=1#HPRTF is the runtime factor when HP PLR is between 0 and 0.2; IGNORING FOR NOW
    P_condenser=((Q_coil*CAPFT_HP_cool)/(COP_cool_ref))*EIRFT_HP_cool*EIRFPLR_cool*HPRTF #Watts
  ##Heating calcs
  if mode=='heat':
    CAPFT_HP_heat=x_ft_heat[0]+x_ft_heat[1]*WB_RA+x_ft_heat[2]*WB_RA**2+x_ft_heat[3]*DB_OA+x_ft_heat[4]*DB_OA**2+x_ft_heat[5]*WB_RA*DB_OA
    Q_heat_available=Q_heat_rated*CAPFT_HP_heat
    PLR_heat=Q_coil/Q_heat_available
    EIRFT_HP_heat=x_EIR_heat[0]+x_EIR_heat[1]*WB_RA+x_EIR_heat[2]*WB_RA**2+x_EIR_heat[3]*DB_OA+x_EIR_heat[4]*DB_OA**2+x_EIR_heat[5]*WB_RA*DB_OA
    EIRFPLR_heat=x_PLR_heat[0]+x_PLR_heat[1]+x_PLR_heat[2]*PLR_heat**2+x_PLR_heat[3]*PLR_heat**3
    CR_rated=Q_coil/Q_heat_rated
  
    CR_heating_corection=1 ##CR_cooling_correction is for when the number of indoor unit connected load exceeds the outdoor units total load, IGNORING FOR NOW
    HPRTF=1#HPRTF is the runtime factor when HP PLR is between 0 and 0.2; IGNORING FOR NOW  
    P_condenser=((Q_coil*CAPFT_HP_heat)/(COP_heat_ref))*EIRFT_HP_heat*EIRFPLR_heat*HPRTF #Watts
  
  return(P_condenser)

  ##DOAS Unit, Hydronic coil and direct evaporative cooling section to maintain set-point
def DOAS(V_dot_DOAS_full, m_dot_DOAS,DB_OA, T_chws,T_hws,T_set_point,Occupancy):
  #AHU Constants
  c1=2.4292  #From Wang need to update, 2004
  c2=0.4978 #From Wang need to update
  c3=0.8     #From Wang need to update
  eff_pump=0.58 #General, need to make a function of flow
  H_chw=33 #meters, assumption to start
  H_hw=33 #meters, assumption to start
  dp_SA=373.623 #Pa, Assuming 1.5" WC
  eff_fan=0.4 #Fan efficiency
  cp_w=4.19 #kj/kgC
  cp_a=1.005 #kJ/kgc
  
  if DB_OA > T_set_point: ##DOAS needs to cool air to acheive set-point
    Q_coil=0 #DOAS is using direct evap which can always meet set-point due to dry weather
    P_fan_DOAS_full=V_dot_DOAS_full*dp_SA/(eff_fan*1000) #kWh, Single fan, fan efficiency can be defined by a curve fitting; Use manufacturer website; Head will need to be an approximation, Janna
    P_fan_DOAS=P_fan_DOAS_full*(Occupancy**3)


  if DB_OA < T_set_point: ##DOAS needs to heat outside air to acheive set-point.  DOAS tied to VRF system
    Q_coil=m_dot_DOAS*cp_a*(T_set_point-DB_OA) #kW
    P_fan_DOAS_full=V_dot_DOAS*dp_SA/(eff_fan*1000) #kWh, Single fan, fan efficiency can be defined by a curve fitting; Use manufacturer website; Head will need to be an approximation, Janna
    P_fan_DOAS=P_fan_DOAS_full*Occupancy**3

  return(P_fan_DOAS, Q_coil)

def VRF_288():
  ###Inputs for 288 sized HP; LG ARUN288
  COP_cool_ref=3.34
  COP_heat_ref=3.41
  Q_cool_rated=84384.0 #W
  Q_heat_rated=94932.0 #W
  IEER_rated=20 #(btu/h)/w

##Cooling Data
  #VRF Cool Cap Boundary
  x_oa_cool=[25.73,-0.03150043,-0.01416595,0]
  #VRF cool capFTHI
  x_ft_cool=[0.6867358,0.0207631,0.0005447,-0.0016218,-4.259e-07,-0.0003392]
  #VRF Cool EIRFTHI
  x_EIR_cool=[-1.4395110176,0.1619850459,-0.0034911781,0.0269442645,0.0001346163,-0.0006714941]
  #VRF EIRPLR hi
  x_PLR_cool=[1,0,0,0]
  #VRFCPLFFPLR
  x_ffplr=[0.85,0.15,0,0]

##Heating Data
  #VRF Heat Cap Boundary
  x_oa_heat=[58.577,-3.0255,0.0193,0]
  #VRF heat capFTHI
  x_ft_heat=[2.5859872368,-0.0953227101,0.0009553288,0,0,0]
  #VRF heat EIRFTHI
  x_EIR_heat=[1.3885703646,-0.0229771462,0.000537274,-0.0273936962,0.0004030426,-5.9786e-05]
  #VRF EIRPLR hi
  x_PLR_heat=[1,0,0,0]
  #VRFCPLFFPLR
  x_ffplr=[0.85,0.15,0,0]
  return(COP_cool_ref,COP_heat_ref,Q_cool_rated,Q_heat_rated,IEER_rated,x_oa_cool,x_ft_cool,x_EIR_cool,x_PLR_cool,x_ffplr,x_oa_heat,x_ft_heat,x_EIR_heat,x_PLR_heat)


# Air Source VRF Annual Energy Model

In [0]:
  #Define Results Database
  columns=['DB_OA','P_fan_DOAS','Cooling_Load-kWh','Electricity_Cost','Q_heating_coil','P_condenser','P_VRF_Indoor']
  results_vrf_air = pd.DataFrame(index=pd.RangeIndex(start=0, stop=8760, step=1), columns=columns)
  #Building_Size=8454.1766 #Square footage of FASB = 91,000 Ft^2
  P_fan_VAV=0

  #General Constants
  T_chws=3.33 #Chilled Water plant runs at 38F
  Min_OA_percentage=.15 #Assumption
  den_air=1.0228 #m^3/kg
  
  T_hws=68.33 #Hot Water supply to building after HX, defined by University
  T_min_SA=12.78
  cp_air=1.005 #kJ/kgc
    
  #Space conditions
  DB_RA=22.22 #Celcius
  WB_RA=15.59 #Celcius
  RH_RA=0.5 #%
  H_RA=48.11 #kj/kg
  W_RA=10.14 #kg/kg  

  #VRF Indoor critera
  P_VRF_Indoor_Unit_Heating=0.04 #kW   http://meus1.mylinkdrive.com/files/PEFY-P08NMAU_Submittal.pdf
  P_VRF_Indoor_Unit_Cooling=0.06 #kW
  Quantity_VRF_Indoor_Units=Building_Size/28 ##Assuming one box per 28 square meters

  for x in range(0,8760):
    #Get Date from WeatherFile
    step_date=weatherfile.loc[x,'Date']
    #Assigns weather conditions at this instance of time
    RH_OA=weatherfile.loc[x,'RH']
    DB_OA=weatherfile.loc[x,'T_DB']
    p_kPa=weatherfile.loc[x,'Atm_Press']
    (H_OA,W_OA,WB_OA,P_SAT_OA)=Psych(DB_OA,RH_OA,p_kPa)
    #Assign Occupancy
    hourly_Occupancy= hourly_Occupancy.set_index(hourly_Occupancy['Date'])
    if (step_date in hourly_Occupancy.index):
      Occupancy=hourly_Occupancy.at[step_date,'Occupancy']

    #Assign airflow
    Occupancy_density=10/100 #people per 100m^2 ASHRAE 62.1
    People_Air_Rate=2.5*0.001 #2.5 l/s * 0.001 m^3/liter ASHRAE 62.1
    Area_Air_Rate=0.3*0.001 #m^3/sm^2  ASHRAE 62.1

    V_dot_DOAS_full=Occupancy_density*Building_Size*People_Air_Rate+Building_Size*Area_Air_Rate
    V_dot_DOAS=Occupancy_density*Occupancy*Building_Size*People_Air_Rate+Building_Size*Area_Air_Rate
    m_dot_DOAS=V_dot_DOAS/den_air

    #Run DOAS to provide conditioned outside air to building as a function of occupancy
    T_set_point=21.67  #DOAS always provides neutral air to the VRF units
    (P_fan_DOAS, Q_heat_DOAS)=DOAS(V_dot_DOAS_full, m_dot_DOAS,DB_OA, T_chws,T_hws,T_set_point,Occupancy)
    
    #Assign Cooling Load at this instance of time
    hourly_load_cool = hourly_load_cool.set_index(hourly_load_cool['Date'])
    if (step_date in hourly_load_cool.index):
      Step=hourly_load_cool.loc[step_date]
      Q_cooling_coil=Step['Cooling_Load']*Building_Size #Finds cooling load as a function of building Size

    
    else:
      Q_cooling_coil=0 #If load database doesn't have a matching date return 0
    
    #Assign Heating Load at this instance of time
    hourly_load_heat = hourly_load_heat.set_index(hourly_load_heat['Date'])
    if (step_date in hourly_load_heat.index):
      Step=hourly_load_heat.loc[step_date]
      Q_heating_coil=Step['Heating_Load']*Building_Size #Finds heating load as a function of building Size;
    
    else:
      Q_heating_coil=0 #If load database doesn't have a matching date return 0
     
    ##Ignore heating load if DB_RA is less than DB_OA
    if DB_RA<DB_OA:
      Q_heating_coil=0  

    # VRF Condenser cool, Assuming 5-condensers; Need to program a leed/lag scheme to turn on and off
    if Q_cooling_coil>Q_heating_coil:
      mode='cool'
 
    if Q_heating_coil>Q_cooling_coil:
      mode='heat'
      
    Quantity_condensers=10 ##Peak load of 700 kW, 84 kW per condenser    
    ##Allows for heat recovery between zones when cooling and heating occurs simultenously
    Q_condenser=abs((Q_heating_coil)-Q_cooling_coil)/Quantity_condensers
    P_condenser=VRF_Condenser(DB_OA,WB_RA,Q_condenser,mode)*Quantity_condensers #kW

    #VRF Indoor Unit Power Consumption
    if mode=='cool':
      P_VRF_Indoor=P_VRF_Indoor_Unit_Cooling*Quantity_VRF_Indoor_Units*Occupancy
    if mode=='heat':
      P_VRF_Indoor=P_VRF_Indoor_Unit_Heating*Quantity_VRF_Indoor_Units*Occupancy
    
    #Calculate Costs per hour
    Total_Electricty=P_fan_DOAS+P_condenser+P_VRF_Indoor
    Electricity_Cost=Total_Electricty*Electricity_rate

    #Write results to database
    results_vrf_air.iloc[x, results_vrf_air.columns.get_loc('DB_OA')] = DB_OA
    results_vrf_air.iloc[x, results_vrf_air.columns.get_loc('P_fan_DOAS')] = P_fan_DOAS
    results_vrf_air.iloc[x, results_vrf_air.columns.get_loc('Cooling_Load-kWh')] = Q_cooling_coil
    results_vrf_air.iloc[x, results_vrf_air.columns.get_loc('Electricity_Cost')] = Electricity_Cost
    results_vrf_air.iloc[x, results_vrf_air.columns.get_loc('Q_heating_coil')] = Q_heating_coil
    results_vrf_air.iloc[x, results_vrf_air.columns.get_loc('P_condenser')] = P_condenser
    results_vrf_air.iloc[x, results_vrf_air.columns.get_loc('P_VRF_Indoor')] = P_VRF_Indoor
  results_vrf_air

  results_vrf_air.to_csv('results_vrf_air.csv')
  

# Cooling Tower Neural Network

In [9]:
#Install Tensorflow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers


print(tf.__version__)

TensorFlow 2.x selected.
2.1.0


In [10]:
#Download NN model from Github
from urllib.request import urlretrieve
import os
from zipfile import ZipFile

def download(url, file):
    if not os.path.isfile(file):
        print("Download file... " + file + " ...")
        urlretrieve(url,file)
        print("File downloaded")
        
download('https://github.com/SSESLab/Campus-Decision-Tool/blob/master/Cooling_Tower_NN.h5?raw=true','model')
print("All the files are downloaded")

Download file... model ...
File downloaded
All the files are downloaded


In [12]:
# Recreate the exact same model, including its weights and the optimizer
CT_model = tf.keras.models.load_model('model')
# Show the model architecture
CT_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                160       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 3,393
Trainable params: 3,393
Non-trainable params: 0
_________________________________________________________________


# Water Source VRF + Cooling tower + Electric Boiler Annual Energy Model

In [0]:
  #Define Results Database
  columns=['DB_OA','P_fan_DOAS','Cooling_Load-kWh','Electricity_Cost','Q_heating_coil','P_condenser','P_VRF_Indoor','P_fan_CT','VFD_speed','T_condr','P_boiler']
  results_vrf_water = pd.DataFrame(index=pd.RangeIndex(start=0, stop=8760, step=1), columns=columns)
  P_fan_VAV=0
 
  #General Constants
  T_chws=3.33 #Chilled Water plant runs at 38F
  Min_OA_percentage=.15 #Assumption
  den_air=1.0228 #m^3/kg
  den_water=997 #kg/m^3
  T_hws=68.33 #Hot Water supply to building after HX, Defined by University
  T_min_SA=12.78
  cp_air=1.005 #kJ/kgc
  cp_w=4.19 #kJ/kg c
    
  #Space conditions
  DB_RA=22.22 #Celcius
  WB_RA=15.59 #Celcius
  RH_RA=0.5 #%
  H_RA=48.11 #kj/kg
  W_RA=10.14 #kg/kg  

  #VRF Indoor critera
  P_VRF_Indoor_Unit_Heating=0.04 #kW   http://meus1.mylinkdrive.com/files/PEFY-P08NMAU_Submittal.pdf
  P_VRF_Indoor_Unit_Cooling=0.06 #kW
  Quantity_VRF_Indoor_Units=Building_Size/28 ##Assuming one box per 28 square meters
  Quantity_condensers=10  

  for x in range(0,8760):
    #Get Date from WeatherFile
    step_date=weatherfile.loc[x,'Date']
    #Assigns weather conditions at this instance of time
    RH_OA=weatherfile.loc[x,'RH']
    DB_OA=weatherfile.loc[x,'T_DB']
    p_kPa=weatherfile.loc[x,'Atm_Press']
    (H_OA,W_OA,WB_OA,P_SAT_OA)=Psych(DB_OA,RH_OA,p_kPa)
    #Assign Occupancy
    hourly_Occupancy= hourly_Occupancy.set_index(hourly_Occupancy['Date'])
    if (step_date in hourly_Occupancy.index):
      Occupancy=hourly_Occupancy.at[step_date,'Occupancy']
    
    #Assign airflow
    Occupancy_density=10/100 #people per 100m^2 ASHRAE 62.1
    People_Air_Rate=2.5*0.001 #2.5 l/s * 0.001 m/liter ASHRAE 62.1
    Area_Air_Rate=0.3*0.001 #m^3/sm^2  ASHRAE 62.1

    V_dot_DOAS_full=Occupancy_density*Building_Size*People_Air_Rate+Building_Size*Area_Air_Rate
    V_dot_DOAS=Occupancy_density*Occupancy*Building_Size*People_Air_Rate+Building_Size*Area_Air_Rate
    m_dot_DOAS=V_dot_DOAS/den_air

    #Run DOAS to provide conditioned outside air to building as a function of occupancy
    T_set_point=21.67  #DOAS always provides neutral air to the VRF units
    (P_fan_DOAS, Q_heat_DOAS)=DOAS(V_dot_DOAS_full, m_dot_DOAS,DB_OA, T_chws,T_hws,T_set_point,Occupancy)
    
    #Assign Cooling Load at this instanace of time
    hourly_load_cool = hourly_load_cool.set_index(hourly_load_cool['Date'])
    if (step_date in hourly_load_cool.index):
      Step=hourly_load_cool.loc[step_date]
      Q_cooling_coil=Step['Cooling_Load']*Building_Size #Finds cooling load as a function of building Size

    
    else:
      Q_cooling_coil=0 #If load database doesn't have a matching date return 0
    
    #Assign Heating Load at this instance of time
    hourly_load_heat = hourly_load_heat.set_index(hourly_load_heat['Date'])
    if (step_date in hourly_load_heat.index):
      Step=hourly_load_heat.loc[step_date]
      Q_heating_coil=Step['Heating_Load']*Building_Size #Finds heating load as a function of building Size;
    
    else:
      Q_heating_coil=0 #If load database doesn't have a matching date return 0
     
    ##Ignore heating load if DB_RA is less than DB_OA
    if DB_RA<DB_OA:
      Q_heating_coil=0  
    
    # VRF Condenser cool, Assuming 5-condensers; Need to program a leed/lag scheme to turn on and off
    if Q_cooling_coil>(Q_heating_coil+Q_heat_DOAS):
      mode='cool'
 
    if (Q_heating_coil+Q_heat_DOAS)>Q_cooling_coil:
      mode='heat'
    
    v_dot_cond=81 #m^3/hr  https://lghvac.com/commercial/product-type/model-details/?productTypeId=a2x44000003XR0i&modelId=01tE0000009NX48&modelNumber=ARWB288BAS4&iscommercial=true&class=Outdoor%20Units
    T_conds=25 #Cooling tower set-point to supply to VRF condenser
    #If mode is heating use DB_OA if cooling use T_conds
    m_dot_cond=v_dot_cond*den_water/3600 #kg/s
    
    
    Quantity_condensers=10 ##Peak load of 700 kW, 84 kW per condenser  
    ##Allows for heat recovery between zones when cooling and heating occurs simultenously
    Q_condenser=abs((Q_heating_coil)-Q_cooling_coil)/Quantity_condensers

    #If mode is heating use boiler to heat condenser water
    if mode=='heat':
      P_condenser=VRF_Condenser(T_conds,WB_RA,Q_condenser,mode)*Quantity_condensers #kW
      Q_boiler=(Q_heating_coil)-P_condenser ##Boiler heat energy is equal to total energy moved by Heat pump minus energy supplied to heat pump
      #Solve for water temperature leaving 
      T_condr=T_conds-Q_boiler/(m_dot_cond*cp_w)

    else: #if cooling use T_conds as the rejection temperature
      P_condenser=VRF_Condenser(T_conds,WB_RA,Q_condenser,mode)*Quantity_condensers #kW
      Q_tower_rej=Q_cooling_coil-P_condenser ##Cooling tower rejection energy is equal to total energy moved by Heat pump minus energy supplied to heat pump
      #Solve for water temperature leaving 
      T_condr=T_conds+Q_tower_rej/(m_dot_cond*cp_w)
    
    
    #VRF Indoor Unit Power Consumption
    if mode=='cool':
      P_VRF_Indoor=P_VRF_Indoor_Unit_Cooling*Quantity_VRF_Indoor_Units*Occupancy
    if mode=='heat':
      P_VRF_Indoor=P_VRF_Indoor_Unit_Heating*Quantity_VRF_Indoor_Units*Occupancy
    
    
    if mode=='cool': # Call cooling tower module to calcuate cooling tower fan speed
      #Organize NN data
      # initialize list of lists 
      data = [[WB_OA,T_condr,T_conds,v_dot_cond]] 
  
      #Fill dataframe with current time step data
      step_CT_data = pd.DataFrame(data, columns = ['WB_OA', 'T_condr','T_conds','V_dot_cond'])

      VFD_speed = CT_model.predict(step_CT_data).item(0)
      P_fan_CT_fullspeed=7.46 #BkW
      P_fan_CT=P_fan_CT_fullspeed*(VFD_speed/100)**3
      P_boiler=0
      if VFD_speed<0: #Fan can't slow down to be less than zero.  Bounding constraint of Neural Network
        VFD_speed=0
        P_fan_CT=0
    if mode=='heat': #Unit is in heating and cooling tower is off
      P_fan_CT=0
      VFD_speed=0
      P_boiler=ElectricBoiler(m_dot_cond,T_conds,T_condr)


    #Calculate Costs per hour
    Total_Electricty=P_fan_DOAS+P_condenser+P_VRF_Indoor+P_fan_CT+P_boiler
    Electricity_Cost=Total_Electricty*Electricity_rate

    
    #Write results to database
    results_vrf_water.iloc[x, results_vrf_water.columns.get_loc('DB_OA')] = DB_OA
    results_vrf_water.iloc[x, results_vrf_water.columns.get_loc('P_fan_DOAS')] = P_fan_DOAS
    results_vrf_water.iloc[x, results_vrf_water.columns.get_loc('Cooling_Load-kWh')] = Q_cooling_coil
    results_vrf_water.iloc[x, results_vrf_water.columns.get_loc('Electricity_Cost')] = Electricity_Cost
    results_vrf_water.iloc[x, results_vrf_water.columns.get_loc('Q_heating_coil')] = Q_heating_coil
    results_vrf_water.iloc[x, results_vrf_water.columns.get_loc('P_condenser')] = P_condenser
    results_vrf_water.iloc[x, results_vrf_water.columns.get_loc('P_VRF_Indoor')] = P_VRF_Indoor
    results_vrf_water.iloc[x, results_vrf_water.columns.get_loc('P_fan_CT')] = P_fan_CT
    results_vrf_water.iloc[x, results_vrf_water.columns.get_loc('VFD_speed')] = VFD_speed
    results_vrf_water.iloc[x, results_vrf_water.columns.get_loc('T_condr')] = T_condr
    results_vrf_water.iloc[x, results_vrf_water.columns.get_loc('P_boiler')] = P_boiler
  results_vrf_water
  results_vrf_water.to_csv('results_vrf_water.csv')


# Water Source VRF + Cooling tower + Gas Boiler Annual Energy Model

In [0]:
  #Define Results Database
  columns=['DB_OA','P_fan_DOAS','Cooling_Load-kWh','Electricity_Cost','Q_heating_coil','P_condenser','P_VRF_Indoor','P_fan_CT','VFD_speed','T_condr','P_boiler','Gas_Cost']
  results_vrf_water_gas = pd.DataFrame(index=pd.RangeIndex(start=0, stop=8760, step=1), columns=columns)
  #Building_Size=8454.1766 #Square footage of FASB = 91,000 Ft^2
  P_fan_VAV=0

  #General Constants
  T_chws=3.33 #Chilled Water plant runs at 38F
  Min_OA_percentage=.15 #Assumption
  den_air=1.0228 #m^3/kg
  den_water=997 #kg/m^3
  T_hws=68.33 #Hot Water supply to building after HX, using Kennecot as example
  T_min_SA=12.78
  cp_air=1.005 #kJ/kgc
  cp_w=4.19 #kJ/kg c
    
  #Space conditions
  DB_RA=22.22 #Celcius
  WB_RA=15.59 #Celcius
  RH_RA=0.5 #%
  H_RA=48.11 #kj/kg
  W_RA=10.14 #kg/kg  

  #VRF Indoor critera
  P_VRF_Indoor_Unit_Heating=0.04 #kW   http://meus1.mylinkdrive.com/files/PEFY-P08NMAU_Submittal.pdf
  P_VRF_Indoor_Unit_Cooling=0.06 #kW
  Quantity_VRF_Indoor_Units=Building_Size/28 ##Assuming one box per 28 square meters
  Quantity_condensers=10 ##Peak load of 700 kW, 84 kW per condenser  

  ##Energy Costs  - From UU Facilities
  Central_Heating_rate=0.015407 #$/kWh heating, UofU gave 4.5 #$/MMBTU, 1 MMbtu=293 kWh, 4.5/293=0.015407

  for x in range(0,8760):
    #Get Date from WeatherFile
    step_date=weatherfile.loc[x,'Date']
    #Assigns weather conditions at this instance of time
    RH_OA=weatherfile.loc[x,'RH']
    DB_OA=weatherfile.loc[x,'T_DB']
    p_kPa=weatherfile.loc[x,'Atm_Press']
    (H_OA,W_OA,WB_OA,P_SAT_OA)=Psych(DB_OA,RH_OA,p_kPa)
    #Assign Occupancy
    hourly_Occupancy= hourly_Occupancy.set_index(hourly_Occupancy['Date'])
    if (step_date in hourly_Occupancy.index):
      Occupancy=hourly_Occupancy.at[step_date,'Occupancy']
   
    #Assign airflow
    Occupancy_density=10/100 #people per 100m^2 ASHRAE 62.1
    People_Air_Rate=2.5*0.001 #2.5 l/s * 0.001 m/liter ASHRAE 62.1
    Area_Air_Rate=0.3*0.001 #m^3/sm^2  ASHRAE 62.1

    V_dot_DOAS_full=Occupancy_density*Building_Size*People_Air_Rate+Building_Size*Area_Air_Rate
    V_dot_DOAS=Occupancy_density*Occupancy*Building_Size*People_Air_Rate+Building_Size*Area_Air_Rate
    m_dot_DOAS=V_dot_DOAS/den_air

    #Run DOAS to provide conditioned outside air to building as a function of occupancy
    T_set_point=21.67  #DOAS always provides neutral air to the VRF units
    (P_fan_DOAS, Q_heat_DOAS)=DOAS(V_dot_DOAS_full, m_dot_DOAS,DB_OA, T_chws,T_hws,T_set_point,Occupancy)
    
    #Assign Cooling Load at this instanace of time
    hourly_load_cool = hourly_load_cool.set_index(hourly_load_cool['Date'])
    if (step_date in hourly_load_cool.index):
      Step=hourly_load_cool.loc[step_date]
      Q_cooling_coil=Step['Cooling_Load']*Building_Size #Finds cooling load as a function of building Size

    
    else:
      Q_cooling_coil=0 #If load database doesn't have a matching date return 0
    
    #Assign Heating Load at this instance of time
    hourly_load_heat = hourly_load_heat.set_index(hourly_load_heat['Date'])
    if (step_date in hourly_load_heat.index):
      Step=hourly_load_heat.loc[step_date]
      Q_heating_coil=Step['Heating_Load']*Building_Size #Finds heating load as a function of building Size; 133 is the average summer gas load, removes OA load from condenser
    
    else:
      Q_heating_coil=0 #If load database doesn't have a matching date return 0
     
    ##Ignore heating load if DB_RA is less than DB_OA
    if DB_RA<DB_OA:
      Q_heating_coil=0  
    
    # VRF Condenser cool, Assuming 5-condensers; Need to program a leed/lag scheme to turn on and off
    if Q_cooling_coil>(Q_heating_coil):
      mode='cool'
 
    if (Q_heating_coil)>Q_cooling_coil:
      mode='heat'
    
    v_dot_cond=81 #m^3/hr  https://lghvac.com/commercial/product-type/model-details/?productTypeId=a2x44000003XR0i&modelId=01tE0000009NX48&modelNumber=ARWB288BAS4&iscommercial=true&class=Outdoor%20Units
    T_conds=25 #Cooling tower set-point to supply to VRF condenser
    #If mode is heating use DB_OA if cooling use T_conds
    m_dot_cond=v_dot_cond*den_water/3600 #kg/s
    
    
    Quantity_condensers=10 ##Peak load of 700 kW, 84 kW per condenser  
    ##Allows for heat recovery between zones when cooling and heating occurs simultenously
    Q_condenser=abs((Q_heating_coil)-Q_cooling_coil)/Quantity_condensers

    #If mode is heating use boiler to heat condenser water
    if mode=='heat':
      P_condenser=VRF_Condenser(T_conds,WB_RA,Q_condenser,mode)*Quantity_condensers #kW
      Q_boiler=(Q_heating_coil)-P_condenser ##Boiler heat energy is equal to total energy moved by Heat pump minus energy supplied to heat pump
      #Solve for water temperature leaving 
      T_condr=T_conds-Q_boiler/(m_dot_cond*cp_w)

    else: #if cooling use T_conds as the rejection temperature
      P_condenser=VRF_Condenser(T_conds,WB_RA,Q_condenser,mode)*Quantity_condensers #kW
      Q_tower_rej=Q_cooling_coil-P_condenser ##Cooling tower rejection energy is equal to total energy moved by Heat pump minus energy supplied to heat pump
      #Solve for water temperature leaving 
      T_condr=T_conds+Q_tower_rej/(m_dot_cond*cp_w)
    
    
    #VRF Indoor Unit Power Consumption
    if mode=='cool':
      P_VRF_Indoor=P_VRF_Indoor_Unit_Cooling*Quantity_VRF_Indoor_Units*Occupancy
    if mode=='heat':
      P_VRF_Indoor=P_VRF_Indoor_Unit_Heating*Quantity_VRF_Indoor_Units*Occupancy
    
    
    if mode=='cool': # Call cooling tower module to calcuate cooling tower fan speed
      #Organize NN data
      # initialize list of lists 
      data = [[WB_OA,T_condr,T_conds,v_dot_cond]] 
  
      #Fill dataframe with current time step data
      step_CT_data = pd.DataFrame(data, columns = ['WB_OA', 'T_condr','T_conds','V_dot_cond'])

      VFD_speed = CT_model.predict(step_CT_data).item(0)
      P_fan_CT_fullspeed=7.46 #BkW
      P_fan_CT=P_fan_CT_fullspeed*(VFD_speed/100)**3
      P_boiler=0
      if VFD_speed<0: #Fan can't slow down to be less than zero.  Bounding constraint of Neural Network
        VFD_speed=0
        P_fan_CT=0
    if mode=='heat': #Unit is in heating and cooling tower is off
      P_fan_CT=0
      VFD_speed=0
      P_boiler=ElectricBoiler(m_dot_cond,T_conds,T_condr)



    #Calculate Costs per hour
    Total_Electricty=P_fan_DOAS+P_condenser+P_VRF_Indoor+P_fan_CT
    Electricity_Cost=Total_Electricty*Electricity_rate

    Gas_Cost=P_boiler*Central_Heating_rate
    
    #Write results to database
    results_vrf_water_gas.iloc[x, results_vrf_water_gas.columns.get_loc('DB_OA')] = DB_OA
    results_vrf_water_gas.iloc[x, results_vrf_water_gas.columns.get_loc('P_fan_DOAS')] = P_fan_DOAS
    results_vrf_water_gas.iloc[x, results_vrf_water_gas.columns.get_loc('Cooling_Load-kWh')] = Q_cooling_coil
    results_vrf_water_gas.iloc[x, results_vrf_water_gas.columns.get_loc('Electricity_Cost')] = Electricity_Cost
    results_vrf_water_gas.iloc[x, results_vrf_water_gas.columns.get_loc('Q_heating_coil')] = Q_heating_coil
    results_vrf_water_gas.iloc[x, results_vrf_water_gas.columns.get_loc('P_condenser')] = P_condenser
    results_vrf_water_gas.iloc[x, results_vrf_water_gas.columns.get_loc('P_VRF_Indoor')] = P_VRF_Indoor
    results_vrf_water_gas.iloc[x, results_vrf_water_gas.columns.get_loc('P_fan_CT')] = P_fan_CT
    results_vrf_water_gas.iloc[x, results_vrf_water_gas.columns.get_loc('VFD_speed')] = VFD_speed
    results_vrf_water_gas.iloc[x, results_vrf_water_gas.columns.get_loc('T_condr')] = T_condr
    results_vrf_water_gas.iloc[x, results_vrf_water_gas.columns.get_loc('P_boiler')] = P_boiler
    results_vrf_water_gas.iloc[x, results_vrf_water_gas.columns.get_loc('Gas_Cost')] = Gas_Cost
  
  results_vrf_water_gas.to_csv('results_vrf_water_gas.csv')


Water Source VRF + Ground Loop

In [0]:
#Define Results Database
  columns=['DB_OA','P_fan_DOAS','Cooling_Load-kWh','Electricity_Cost','Q_heating_coil','P_condenser','P_VRF_Indoor']
  results_vrf_ground = pd.DataFrame(index=pd.RangeIndex(start=0, stop=8760, step=1), columns=columns)

  Building_Size=11148 #Square Meter, 120,000 SF per wing
  P_fan_VAV=0
 
  #General Constants
  T_chws=3.33 #Chilled Water plant runs at 38F
  Min_OA_percentage=.15 #Assumption
  den_air=1.0228 #m^3/kg
  den_water=997 #kg/m^3
  T_hws=68.33 #Hot Water supply to building after HX, using Kennecot as example
  T_min_SA=12.78
  cp_air=1.005 #kJ/kgc
  cp_w=4.19 #kJ/kg c

  # Well Temperature
  T_well_cooling=12.22 #C
  T_well_heating=8.33 #C
    
  #Space conditions
  DB_RA=22.22 #Celcius
  WB_RA=15.59 #Celcius
  RH_RA=0.5 #%
  H_RA=48.11 #kj/kg
  W_RA=10.14 #kg/kg  

  #VRF Indoor critera
  P_VRF_Indoor_Unit_Heating=0.04 #kW   http://meus1.mylinkdrive.com/files/PEFY-P08NMAU_Submittal.pdf
  P_VRF_Indoor_Unit_Cooling=0.06 #kW
  Quantity_VRF_Indoor_Units=Building_Size/28 ##Assuming one box per 28 square meters
  Quantity_condensers=10 ##Peak load of 700 kW, 84 kW per condenser  

  for x in range(0,8760):
    #Get Date from WeatherFile
    step_date=weatherfile.loc[x,'Date']
    #Assigns weather conditions at this instance of time
    RH_OA=weatherfile.loc[x,'RH']
    DB_OA=weatherfile.loc[x,'T_DB']
    p_kPa=weatherfile.loc[x,'Atm_Press']
    (H_OA,W_OA,WB_OA,P_SAT_OA)=Psych(DB_OA,RH_OA,p_kPa)
    #Assign Occupancy
    hourly_Occupancy= hourly_Occupancy.set_index(hourly_Occupancy['Date'])
    if (step_date in hourly_Occupancy.index):
      Occupancy=hourly_Occupancy.at[step_date,'Occupancy']
   
    #Assign airflow
    Occupancy_density=10/100 #people per 100m^2 ASHRAE 62.1
    People_Air_Rate=2.5*0.001 #2.5 l/s * 0.001 m/liter ASHRAE 62.1
    Area_Air_Rate=0.3*0.001 #m^3/sm^2  ASHRAE 62.1

    V_dot_DOAS_full=Occupancy_density*Building_Size*People_Air_Rate+Building_Size*Area_Air_Rate
    V_dot_DOAS=Occupancy_density*Occupancy*Building_Size*People_Air_Rate+Building_Size*Area_Air_Rate
    m_dot_DOAS=V_dot_DOAS/den_air

    #Run DOAS to provide conditioned outside air to building as a function of occupancy
    T_set_point=21.67  #DOAS always provides neutral air to the VRF units
    (P_fan_DOAS, Q_heat_DOAS)=DOAS(V_dot_DOAS_full, m_dot_DOAS,DB_OA, T_chws,T_hws,T_set_point,Occupancy)
    
    #Assign Cooling Load at this instanace of time
    hourly_load_cool = hourly_load_cool.set_index(hourly_load_cool['Date'])
    if (step_date in hourly_load_cool.index):
      Step=hourly_load_cool.loc[step_date]
      Q_cooling_coil=Step['Cooling_Load']*Building_Size #Finds cooling load as a function of building Size

    
    else:
      Q_cooling_coil=0 #If load database doesn't have a matching date return 0
    
    #Assign Heating Load at this instance of time
    hourly_load_heat = hourly_load_heat.set_index(hourly_load_heat['Date'])
    if (step_date in hourly_load_heat.index):
      Step=hourly_load_heat.loc[step_date]
      Q_heating_coil=Step['Heating_Load']*Building_Size #Finds heating load as a function of building Size; 
    
    else:
      Q_heating_coil=0 #If load database doesn't have a matching date return 0
     
    ##Ignore heating load if DB_RA is less than DB_OA
    if DB_RA<DB_OA:
      Q_heating_coil=0  
    
    # VRF Condenser cool, Assuming 5-condensers; Need to program a leed/lag scheme to turn on and off
    if Q_cooling_coil>(Q_heating_coil):
      mode='cool'
 
    if (Q_heating_coilS)>Q_cooling_coil:
      mode='heat'
    
    v_dot_cond=81 #m^3/hr  https://lghvac.com/commercial/product-type/model-details/?productTypeId=a2x44000003XR0i&modelId=01tE0000009NX48&modelNumber=ARWB288BAS4&iscommercial=true&class=Outdoor%20Units
    T_conds=25 #Cooling tower set-point to supply to VRF condenser
    #If mode is heating use DB_OA if cooling use T_conds
    m_dot_cond=v_dot_cond*den_water/3600 #kg/s
    
    
    Quantity_condensers=10 ##Peak load of 700 kW, 84 kW per condenser  
    ##Allows for heat recovery between zones when cooling and heating occurs simultenously
    Q_condenser=abs((Q_heating_coil)-Q_cooling_coil)/Quantity_condensers

    #If mode is heating
    if mode=='heat':
      P_condenser=VRF_Condenser(T_well_heating,WB_RA,Q_condenser,mode)*Quantity_condensers #kW
  
    else: #if mode is in cooling
      P_condenser=VRF_Condenser(T_well_cooling,WB_RA,Q_condenser,mode)*Quantity_condensers #kW   
    
    #VRF Indoor Unit Power Consumption
    if mode=='cool':
      P_VRF_Indoor=P_VRF_Indoor_Unit_Cooling*Quantity_VRF_Indoor_Units*Occupancy
    if mode=='heat':
      P_VRF_Indoor=P_VRF_Indoor_Unit_Heating*Quantity_VRF_Indoor_Units*Occupancy
    
    #Calculate Costs per hour
    Total_Electricty=P_fan_DOAS+P_condenser+P_VRF_Indoor
    Electricity_Cost=Total_Electricty*Electricity_rate
    
    #Write results to database
    results_vrf_ground.iloc[x, results_vrf_ground.columns.get_loc('DB_OA')] = DB_OA
    results_vrf_ground.iloc[x, results_vrf_ground.columns.get_loc('P_fan_DOAS')] = P_fan_DOAS
    results_vrf_ground.iloc[x, results_vrf_ground.columns.get_loc('Cooling_Load-kWh')] = Q_cooling_coil
    results_vrf_ground.iloc[x, results_vrf_ground.columns.get_loc('Electricity_Cost')] = Electricity_Cost
    results_vrf_ground.iloc[x, results_vrf_ground.columns.get_loc('Q_heating_coil')] = Q_heating_coil
    results_vrf_ground.iloc[x, results_vrf_ground.columns.get_loc('P_condenser')] = P_condenser
    results_vrf_ground.iloc[x, results_vrf_ground.columns.get_loc('P_VRF_Indoor')] = P_VRF_Indoor
  
  results_vrf_ground.to_csv('results_vrf_ground.csv')
